RAIN - Real & Artificial Intelligence for Neuroscience

## Geometric analysis
- This notebook will take the position.csv files (created using 1-Prepare_positions.ipynb) and process them using geometric criteria (distance, angle and overall movement)

#### Requirements:
- The position.csv files
- Geometric thresholds for exploration and freezing (see below)

---
#### Load the necessary modules

In [1]:
import os
from glob import glob
import rainstorm.geometric_analysis as rst

---
#### 1. State your project path & thresholds
`base` : The path to the downloaded repository. If you are using a Windows path with backslashes, place an ‘r’ in front of the directory path to avoid an error (e.g. r'C:\Users\dhers\Desktop\RAINSTORM').

`objects` : Name the stationary objects that may appear on your data.

`distance` : Maximum nose-object distance to consider exploration.

`angle` : Maximum head-object orientation angle to consider exploration.

`freezing_threshold` : Movement under which we consider freezing, calculated as the mean standard deviation for all bodyparts over one second.

`darting_threshold` : Movement over which we consider darting, calculated as the mean standard deviation for all bodyparts over one second.

`video_fps` : State the frames per second of the videos.

In [2]:
# State your path:
base = r'C:\Users\dhers\Desktop\RAINSTORM'
# folder_path = os.path.join(base, r'docs\examples\NOR_example')
folder_path = r'C:\Users\dhers\Desktop\SP_6mth'
params = os.path.join(folder_path, 'params.yaml')

---
#### 2. We can open an example file and see what is inside

In [8]:
# Choose an example file to plot:
all_position = glob(os.path.join(folder_path,"*/position/*position.csv"))
example_path = rst.choose_example(all_position, look_for = 'TR1')

Plotting coordinates from 2025_02-R22-Social_TR1_position.csv


In [9]:
# Plot mouse exploration around multiple objects
rst.plot_position(params, example_path)

On the plot we have the position of each object in the example file. We can see the nose tracked and colored according to the angle of approach towards the objects.

We label exploration events when the nose is colored (heading towards the object) and inside the dashed line (close to the object).

---
#### 3.  We can also measure other behaviours, such as freezing

In [10]:
# Plot freezing events
rst.plot_freezing(params, example_path)

Now we are plotting the movement of the mouse. by setting a threshold (we can modify it above) we can quantify the time the mouse spent freezing.

---
#### 4. Finally, we can analyze the position files and create:
- A movement.csv file containing distance traveled, freezing and darting.
- A geolabels.csv file with the exploration of each object, frame by frame.

In [6]:
# Analyze all files in the folder
rst.create_movement_and_geolabels(params)

Saved geolabels to 2025_02-R01-Social_Hab_geolabels.csv
Saved movement to 2025_02-R01-Social_Hab_movement.csv
Saved geolabels to 2025_02-R02-Social_Hab_geolabels.csv
Saved movement to 2025_02-R02-Social_Hab_movement.csv
Saved geolabels to 2025_02-R03-Social_Hab_geolabels.csv
Saved movement to 2025_02-R03-Social_Hab_movement.csv
Saved geolabels to 2025_02-R04-Social_Hab_geolabels.csv
Saved movement to 2025_02-R04-Social_Hab_movement.csv
Saved geolabels to 2025_02-R05-Social_Hab_geolabels.csv
Saved movement to 2025_02-R05-Social_Hab_movement.csv
Saved geolabels to 2025_02-R06-Social_Hab_geolabels.csv
Saved movement to 2025_02-R06-Social_Hab_movement.csv
Saved geolabels to 2025_02-R07-Social_Hab_geolabels.csv
Saved movement to 2025_02-R07-Social_Hab_movement.csv
Saved geolabels to 2025_02-R08-Social_Hab_geolabels.csv
Saved movement to 2025_02-R08-Social_Hab_movement.csv
Saved geolabels to 2025_02-R09-Social_Hab_geolabels.csv
Saved movement to 2025_02-R09-Social_Hab_movement.csv
Saved geol

---
---
#### Two new csv files were created for each video, one for geolabels and the other for movement related scores.
We can:
- Move on to the next notebook to create an artificial neural network that labels exploratory behavior, 3-Create_models.ipynb
- Loose all patience, go straight to 7-Seize_labels.ipynb and use the geolabels to plot the results of the experiment

---
RAINSTORM - Created on Oct 26, 2023 - @author: Santiago D'hers